## Base de datos
### La base de datos contiene dos tablas, limitado con 400 filas
<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">columna</th>
<th>tipo de dato</th>
<th>significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Nombre del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Plataforma del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Distribuidora del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Desarrollador del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Número de copias vendidas(millones)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Año de lanzamiento</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">columna</th>
<th>tipo de dato</th>
<th>significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Nombre del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td>Puntuación de los críticos según Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td>Puntuación de los usuarios según Metacritic</td>
</tr>
</tbody>
</table>

# 1. Los diez videojuegos más vendidos
<p><img src="https://assets.datacamp.com/production/project_1413/img/video_game.jpg" alt="A video game player choosing a game to play on Nintendo Switch." width="400"></p>
<p>Photo by <a href="https://unsplash.com/@retromoon">Dan Schleusser</a> on <a href="https://unsplash.com">Unsplash</a>.</p>

<p>Encontrar los diez videojuegos mas vendidos y ordenar los resultados de mayor a menor </p>

In [8]:
%%sql

postgresql:///games

-- Seleccionar toda la informacion sobre las ventas de videojuegos
SELECT * FROM game_sales
-- Ordenar los resultados de mayor a menor
ORDER BY games_sold DESC
-- Limitar la busqueda a los diez primeros resultados
LIMIT 10;

10 rows affected.


game,platform,publisher,developer,games_sold,year
Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


#f

## 2. Buscar datos faltantes en "Review"
<p>Encontrar datos faltantes al unir ambas tablas y averiguar que videojuegos no contienen reseñas </p>

In [10]:
%%sql 

-- Contar el numero de videojuegos vendidos
SELECT COUNT(game_sales.game) FROM game_sales LEFT JOIN Reviews
-- Unir la tabla game_sales con reviews
ON game_sales.game=Reviews.game 
--Encontrar los datos vacios de las columnas both critic_score y user_score 
WHERE Reviews.critic_score ISNULL AND Reviews.user_score ISNULL

 * postgresql:///games
1 rows affected.


count
31


## 3. Mejores años de los videojuegos segun los criticos 
<p>Encontrar los diez videojuegos con mejor puntuacion segun los criticos y ordenar los años de mayor a menor </p>

In [12]:
%%sql

-- Seleccionar el año de lanzamiento y la puntuacion promedio de las reseñas de cada año, redondeando y nombrando como avg_critic_score
Select year, ROUND(AVG(critic_score),2) as avg_critic_score FROM game_sales
-- Unir la tabla con game_sales
INNER JOIN reviews ON game_sales.game=reviews.game 
-- Agrupar por año de lanzamiento
GROUP BY year 
-- Ordenar los datos de mayor a menor
ORDER BY avg_critic_score DESC 
-- Limitar a diez resultados
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
1998,9.32
2020,9.20
1993,9.10
1995,9.07
2004,9.03
1982,9.00
2002,8.99
1999,8.93


## 4. Filtrar los videojuegos con mejor puntuacion segun los criticos
<p>Encontrar los diez videojuegos con mejor puntuacion segun los criticos y ordenar los años de mayor a menor </p>
<p>Condicinar los años con mas de cuatro juegos publicados</p>

In [14]:
%%sql 

-- Seleccionar el año de lanzamiento y la puntuacion promedio de las reseñas de cada año, redondeando y nombrando como avg_critic_score
-- Recuento de los videojuegos publicados como num_games
Select g.year, ROUND(AVG(r.critic_score),2) as avg_critic_score, COUNT(g.game) as num_games 
-- Unir la tabla con game_sales
FROM game_sales g INNER JOIN reviews r
ON g.game=r.game 
-- Agrupar por año de lanzamiento
GROUP BY g.year
--Filtrar recuento con mas de cuatro videojuegos publicados
HAVING COUNT(g.game) > 4
-- Ordenar los datos de mayor a menor
ORDER BY avg_critic_score DESC 
-- Limitar a diez resultados
LIMIT 10;


 * postgresql:///games
10 rows affected.


year,avg_critic_score,num_games
1998,9.32,10
2004,9.03,11
2002,8.99,9
1999,8.93,11
2001,8.82,13
2011,8.76,26
2016,8.67,13
2013,8.66,18
2008,8.63,20
2012,8.62,12


### Creación de nuevas tablas
<p>De los resultados obtenidos, se crearon dos tablas: top_critic_years y top_critic_year_more_than__four_games</p>
<h3 id="top_critic_years"><code>top_critic_years</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">columna</th>
<th>tipo de dato</th>
<th>significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Año de lanzamiento del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Puntacion promedio de todas las reseñas de los videojuegos publicados en ese año</td>
</tr>
</tbody>
</table>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">columna</th>
<th>tipo de dato</th>
<th>significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Año de lanzamiento del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Número total de videojuegos lanzados en ese año</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Puntacion promedio de todas las reseñas de videojuegos en ese año</td>
</tr>
</tbody>
</table>

## 5. Años que salieron de la 'lista de videojuegos favoritos' por criticos
<p>Encontrar los años que estan fuera de la lista de favoritos por tener cuatro o menos juegos con reseñas</p>


In [16]:
%%sql 

-- Seleccionar avg_critic_score y el año de lanzamiento
-- Comparar ambas tablas, separando los datos que existan en la primera tabla pero no en la segunda
SELECT year, avg_critic_score
FROM top_critic_years
EXCEPT
SELECT year, avg_critic_score
FROM top_critic_years_more_than_four_games
-- Ordenar los resultados de mayor a menor
ORDER BY avg_critic_score DESC

 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


## 6. Mejores años para los videojuegos segun los jugadores
<p>Encontrar los diez videojuegos con mejor puntuacion segun los jugadores y ordenar los años de mayor a menor</p>
<p>Condicinar cada año con mas de cuatro juegos publicados</p>

In [18]:
%%sql 


-- Seleccionar el año de lanzamiento y la puntuacion promedio de las usuarios en cada año, redondeando y nombrando como avg_user_score
-- Recuento de los videojuegos publicados con un alias y redondeo
Select g.year, ROUND(AVG(r.user_score),2) as avg_user_score, COUNT(g.game) as num_games 
-- Unir ambas tablas
FROM game_sales g INNER JOIN reviews r
ON g.game=r.game 
-- Agrupar por año de lanzamiento
GROUP BY g.year
-- Filtrar con mas de cuatro juegos publicados
HAVING COUNT(g.game) > 4
-- Ordenar de mayor a menor 
ORDER BY avg_user_score DESC 
-- Limitar a diez resultados
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,avg_user_score,num_games
1997,9.50,8
1998,9.40,10
2010,9.24,23
2009,9.18,20
2008,9.03,20
1996,9.00,5
2005,8.95,13
2006,8.95,16
2000,8.80,8
1999,8.80,11


### Creación de nuevas tablas
<p>De los resultados obtenidos, se creo la tabla: top_user_years_more_than_four_games</p>
<h3 id="top_user_years_more_than_four_games"><code>top_user_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">columna</th>
<th>tipo de dato</th>
<th>significado</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Año de lanzamiento del videojuego</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Número total de videojuegos lanzados en ese año</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_user_score</code></td>
<td>float</td>
<td>Puntacion promedio de todos los usuarios de videojuegos en ese año</td>
</tr>
</tbody>
</table>

## 7. Años donde los videojuegos fueron amados por criticos y jugadores
<p>Encontrar los años que aparecen en ambas tablas(dos ultimas tablas)</p>



In [20]:
%%sql 

-- Seleccionar los resultados de los años que pertenecen a ambas tablas 
SELECT year FROM
top_critic_years_more_than_four_games 
INTERSECT
SELECT year FROM
top_user_years_more_than_four_games

 * postgresql:///games
3 rows affected.


year
1998
2008
2002


## 8. Mejores ventas anuales de videojuegos
<p>Encontrar las ventas anuales mas altas de los videojuegos </p>


In [22]:
%%sql 

--Seleccionar el año y el total de venta de copias vendidas
SELECT year, SUM(games_sold) AS total_games_sold FROM game_sales g
-- Filtrar las ventas segun la consulta anterior 
WHERE year IN(SELECT year FROM
top_critic_years_more_than_four_games 
INTERSECT
SELECT year FROM
top_user_years_more_than_four_games)
-- Agrupar por año de lanzamiento
GROUP BY g.year
-- Ordenar los resultados de mayor a menor
ORDER BY total_games_sold DESC

 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67
